In [2]:
!pip install evaluate
!pip install optuna
!pip install datasets
!pip install bert_score
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" an

In [3]:
import json
import torch
from datasets import load_dataset
import nltk
import optuna
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from evaluate import load

# Download NLTK data for sentence tokenization
nltk.download("punkt")

# Step 1: Load the ClimateFever dataset using Hugging Face datasets
print("Loading ClimateFever dataset...")
climatefever_dataset = load_dataset("climate_fever", split="test")

# Step 2: Extract and adapt (problem, approach) pairs
dataset = []

env_keywords = [
    "climate change", "carbon emission", "pollution", "biodiversity",
    "deforestation", "renewable energy", "sustainability", "ocean acidification"
]

def is_env_science(claim):
    claim_lower = claim.lower()
    return any(keyword in claim_lower for keyword in env_keywords)

def synthesize_approach(claim, evidence_list):
    evidence_text = " ".join([evidence["evidence"] for evidence in evidence_list])
    if "carbon emission" in claim.lower() or "global warming" in claim.lower():
        approach = f"To address the issue of {claim.lower()}, a multi-step strategy can be implemented: 1. Promote renewable energy adoption by offering incentives such as tax credits for solar and wind energy installations. 2. Expand public transportation systems to reduce reliance on fossil fuel-based vehicles, especially in urban areas. 3. Implement stricter regulations on industrial emissions, requiring companies to adopt cleaner technologies and report emissions annually. Additionally, public awareness campaigns can educate communities about sustainable practices, such as reducing energy consumption and supporting green policies. International collaboration with organizations like the UN can help secure funding and coordinate efforts across countries, ensuring a unified approach to tackling this issue. {evidence_text} This approach aims to mitigate the environmental impact while fostering long-term sustainability."
    elif "pollution" in claim.lower():
        approach = f"To mitigate {claim.lower()}, a comprehensive plan can be adopted: 1. Enforce regulations banning single-use plastics and promoting biodegradable alternatives. 2. Enhance waste management systems by increasing recycling facilities and ensuring proper disposal in affected regions. 3. Launch cleanup initiatives, such as deploying technologies to remove debris from ecosystems. 4. Educate communities about the impact of pollution through school programs and media campaigns, encouraging reduced waste production. Collaboration with global organizations can help secure funding and coordinate efforts across regions, ensuring a unified approach to tackling this issue. {evidence_text} This strategy aims to reduce pollution while promoting sustainable practices."
    else:
        approach = f"To address {claim.lower()}, the following approach can be implemented: 1. Develop policies to protect ecosystems, such as establishing protected areas and regulating resource extraction. 2. Promote sustainable practices among communities through education and incentives. 3. Invest in research to better understand the issue and develop innovative solutions. 4. Foster international cooperation to address global aspects of the problem. {evidence_text} This approach seeks to balance environmental protection with sustainable development, ensuring long-term benefits for both nature and society."

    word_count = len(approach.split())
    if not (150 <= word_count <= 300):
        if word_count < 150:
            approach += " Furthermore, engaging stakeholders at all levels—from local communities to international policymakers—ensures that solutions are both practical and widely supported, maximizing their impact over time."
        elif word_count > 300:
            approach = " ".join(approach.split()[:300])

    return approach

claim_to_evidence = {}
for entry in climatefever_dataset:
    claim = entry["claim"]
    evidence = entry["evidences"]
    if not is_env_science(claim):
        continue
    if claim not in claim_to_evidence:
        claim_to_evidence[claim] = []
    claim_to_evidence[claim].extend(evidence)

for claim, evidence_list in claim_to_evidence.items():
    if not evidence_list:
        continue
    approach = synthesize_approach(claim, evidence_list)
    dataset.append({"problem": claim, "approach": approach})
    if len(dataset) >= 500:
        break

with open("environmental_science_climatefever_dataset.json", "w") as f:
    json.dump(dataset, f, indent=4)

print(f"Dataset created with {len(dataset)} pairs. Saved to environmental_science_climatefever_dataset.json")

with open("environmental_science_climatefever_dataset.json", "r") as f:
    data = json.load(f)

inputs = ["problem: " + item["problem"] for item in data]
targets = [item["approach"] for item in data]

dataset = Dataset.from_dict({"input_text": inputs, "target_text": targets})

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    training_args = TrainingArguments(
        output_dir=f"./bart_env_science_trial_{trial.number}",
        eval_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_dir=f"./logs/trial_{trial.number}",
        logging_steps=10,
        report_to="none",
    )

    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    trainer.train()

    rouge = load("rouge")
    predictions = []
    references = []

    for example in eval_dataset:
        input_text = example["input_text"]
        inputs = tokenizer(input_text, return_tensors="pt", max_length=64, truncation=True)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        outputs = model.generate(inputs["input_ids"], max_length=256, num_beams=4, early_stopping=True)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(generated)
        references.append(example["target_text"])

    rouge_results = rouge.compute(predictions=predictions, references=references)
    return rouge_results["rougeL"]

print("Starting hyperparameter optimization with Optuna...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

best_trial = study.best_trial
print("Best trial:")
print(f"  ROUGE-L: {best_trial.value}")
print("  Best hyperparameters: ", best_trial.params)

best_learning_rate = best_trial.params["learning_rate"]
best_batch_size = best_trial.params["batch_size"]
best_num_train_epochs = best_trial.params["num_train_epochs"]

final_training_args = TrainingArguments(
    output_dir="./bart_env_science_final",
    eval_strategy="epoch",
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_train_epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs/final",
    logging_steps=10,
)

final_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Training final model with best hyperparameters...")
final_trainer.train()

final_model.save_pretrained("./bart_env_science_final_model")
tokenizer.save_pretrained("./bart_env_science_final_model")

print("Final model training complete and saved to ./bart_env_science_final_model")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loading ClimateFever dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.09k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/869k [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sqp6023 (sqp6023-penn-state) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.716600,1.959965
2,1.844700,1.794456
3,1.228000,1.795447
4,1.034400,1.815832
5,0.776000,1.922184
6,0.572600,1.964445
7,0.470300,2.027740
8,0.367100,2.034924
9,0.315800,2.067180
10,0.274600,2.065202


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Final model training complete and saved to ./bart_env_science_final_model


In [5]:
# Step 6: Evaluate the final model
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load metrics
rouge = load("rouge")
bertscore = load("bertscore")

# Experiment 1: Standard input format
predictions_standard = []
references = []

for example in eval_dataset:
    input_text = example["input_text"]
    inputs = tokenizer(input_text, return_tensors="pt", max_length=64, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    outputs = final_model.generate(inputs["input_ids"], max_length=256, num_beams=4, early_stopping=True)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_standard.append(generated)
    references.append(example["target_text"])

# ✅ Experiment 2: Keyword-augmented format
predictions_keywords = []

for example in eval_dataset:
    problem_text = example["input_text"].replace("problem: ", "")
    input_text_with_keywords = f"problem: {problem_text} [climate change, sustainability]"
    inputs = tokenizer(input_text_with_keywords, return_tensors="pt", max_length=64, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    outputs = final_model.generate(inputs["input_ids"], max_length=256, num_beams=4, early_stopping=True)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_keywords.append(generated)

# Compute ROUGE-L and BERTScore for input with keywords
rouge_results_keywords = rouge.compute(predictions=predictions_keywords, references=references)
bertscore_results_keywords = bertscore.compute(predictions=predictions_keywords, references=references, lang="en")
print("\nEvaluation with keywords in input format:")
print("ROUGE-L:", rouge_results_keywords["rougeL"])
print("BERTScore (F1):", sum(bertscore_results_keywords["f1"]) / len(bertscore_results_keywords["f1"]))

# Manual evaluation: Print a few examples
print("\nManual Evaluation (First 3 Examples):")
for i in range(min(3, len(eval_dataset))):
    print(f"\nProblem: {eval_dataset[i]['input_text']}")
    print(f"Generated Approach (Standard): {predictions_standard[i]}")
    print(f"Generated Approach (With Keywords): {predictions_keywords[i]}")
    print(f"Ground Truth: {references[i]}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Evaluation with keywords in input format:
ROUGE-L: 0.47465975289251106
BERTScore (F1): 0.8966621789667342

Manual Evaluation (First 3 Examples):

Problem: problem: During a state House debate on a jobs and energy bill this week, Democrats offered an amendment that would put the Legislature on record saying that climate change is real and that humans are causing it.
Generated Approach (Standard): To address during a state house debate on a jobs and energy bill this week, Democrats offered an amendment that would put the Legislature on record saying that climate change is real and that humans are causing it., the following approach can be implemented: 1. Develop policies to protect ecosystems, such as establishing protected areas and regulating resource extraction. 2. Promote sustainable practices among communities through education and incentives. 3. Invest in research to better understand the issue and develop innovative solutions. 4. Foster international cooperation to address global